<a href="https://colab.research.google.com/github/ozeiasgodoy/notebooks/blob/main/Pnad_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# O notebook contém a instalação do Spark(para trabalho com grande volume de dados) no Colab

#Carrega dados da PNAD 2020 e vai responder a seguintes questões:


 * Média de renda 
 * Média de renda das pessoas que residem no DF
 * Qual é o estado brasileiro que possui a menor média de renda e valor dessa média
 * Quais os cincos estados brasileiro que possuem as mesnore média de escolaridade e quais são essas respectivas médias
 * Qual é a média de escolaridade entre as mulheres que moram no Paraná e estão entre 25 e 30 anos? 
 * Qual é a média de renda para as pessoas na região Sul do país, que estão na força de trabalho e possuem entre 25 e 35 anos? 
 * Qual é a renda média das mulheres que residem na região Norte do Brasil, possuem graduação, tem idade entre 25 e 35 anos e são pretas ou pardas? 
 * Qual é a renda médiada região sudeste? 






In [1]:
!pip install spark

In [2]:
!pip install pyspark

In [3]:
!pip install findspark

In [4]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from os.path import abspath

from pyspark.sql.types import DoubleType
from pyspark.sql.functions import *
from pyspark.sql.functions import regexp_replace

import pandas as pd
import numpy as np
import zipfile
import requests
from io import BytesIO
import os

In [5]:
#Criar diretorio para armazenar os dados
os.makedirs('./pnad', exist_ok=True)

In [6]:

#Define a url
url = "https://github.com/ozeiasgodoy/notebooks/blob/main/dados/pnad.zip?raw=true"

#Realiza o download
filebytes = BytesIO(
    #stream serve para refazer a conexão, caso o servidor corte a mesma
    requests.get(url, stream=True).content
)


In [7]:
#Extrair o conteudo do zip
myzip = zipfile.ZipFile(filebytes)
myzip.extractall("./pnad")

In [8]:
#Criando a sessão no SPARK
spark = SparkSession.builder.getOrCreate()

In [9]:
#Criando nosso dataFrame
pnad = (
    spark
    .read
    .format("csv")
    .options(header=True, delimiter=',', enconding='latin1', inferSchema=True)
    .load('./pnad/pnad.csv')
)

In [10]:
#Verificando as 5 primeiras linhas do DataFrame
pnad.head(5)

[Row(_id='600ce0685daa95c6616007bb', ano=2020, trimestre=3, uf='Acre', sexo='Homem', idade=6, cor='Parda', graduacao='null', trab='null', ocup='null', renda='null', horastrab='null', anosesco='0'),
 Row(_id='600ce0685daa95c6616007b1', ano=2020, trimestre=3, uf='Acre', sexo='Mulher', idade=35, cor='Parda', graduacao='null', trab='Pessoas na força de trabalho', ocup='Pessoas ocupadas', renda='800', horastrab='36', anosesco='16'),
 Row(_id='600ce0685daa95c6616007cc', ano=2020, trimestre=3, uf='Acre', sexo='Homem', idade=49, cor='Parda', graduacao='null', trab='Pessoas na força de trabalho', ocup='Pessoas ocupadas', renda='9500', horastrab='40', anosesco='15'),
 Row(_id='600ce0685daa95c6616007ba', ano=2020, trimestre=3, uf='Acre', sexo='Homem', idade=10, cor='Parda', graduacao='null', trab='null', ocup='null', renda='null', horastrab='null', anosesco='2'),
 Row(_id='600ce0685daa95c6616007e3', ano=2020, trimestre=3, uf='Acre', sexo='Mulher', idade=15, cor='Parda', graduacao='null', trab='Pe

In [11]:
#Verificando o tamanho do nosso dataframe
pnad.toPandas().shape

(368210, 13)

In [12]:
#Verificando os tipos de dados das colunas
pnad.dtypes

[('_id', 'string'),
 ('ano', 'int'),
 ('trimestre', 'int'),
 ('uf', 'string'),
 ('sexo', 'string'),
 ('idade', 'int'),
 ('cor', 'string'),
 ('graduacao', 'string'),
 ('trab', 'string'),
 ('ocup', 'string'),
 ('renda', 'string'),
 ('horastrab', 'string'),
 ('anosesco', 'string')]

In [13]:
#Convertendo as colunas renda e horastrab para double. E a coluna anosesco para int
pnad = (
    pnad
    .withColumn("renda", f.col("renda").cast("double"))
    .withColumn("horastrab", f.col("horastrab").cast("double"))
    .withColumn("anosesco", f.col("anosesco").cast("int"))
    .dropna()
 )


# 1 - Média de renda

In [14]:
(
    pnad
    .select('renda')
    .agg(avg('renda').alias('media'))
    .toPandas()
)

,media
0,2158.85525


# 2 -  Média de renda das pessoas que residem no DF

In [15]:
(
    pnad
    .select('renda', 'uf')
    .filter(f.col('uf') == 'Distrito Federal')
    .groupby('uf').agg(avg('renda').alias('media'))
    .toPandas()
)

,uf,media
0,Distrito Federal,3780.269062


# 3 - Qual é o estado brasileiro que possui a menor média de renda e valor dessa média

In [16]:
(
    pnad
    .select('renda', 'uf')
    .groupby('uf').agg(avg('renda').alias('media'))
    .orderBy(f.col('media').asc())
    .limit(1)
    .toPandas()
)

,uf,media
0,Maranhão,1203.782856


# 4 - Quais os cincos estados brasileiro que possuem as menores média de escolaridade e quais são essas respectivas médias


In [17]:
(
    pnad
    .select('anosesco', 'uf')
    .groupby('uf').agg(avg('anosesco').alias('media'))
    .orderBy(f.col('media').asc())
    .limit(5)
    .toPandas()
)

,uf,media
0,Bahia,9.420097
1,Piauí,9.626410
2,Maranhão,9.725763
3,Alagoas,9.879549
4,Ceará,9.965234


# 5 - Qual é a média de escolaridade entre as mulheres que moram no Paraná e estão entre 25 e 30 anos? 

In [18]:
(
    pnad
    .select('anosesco', 'uf', 'idade')
    .filter(((f.col('idade') >= 25) & (f.col('idade') <= 30)))
    .filter(f.col('uf') == 'Paraná')
    .filter(f.col('sexo') == 'Mulher')
    .groupby('uf').agg(avg('anosesco').alias('media'))
    .toPandas()
)

,uf,media
0,Paraná,13.45


# Qual é a média de renda para homens e mulheres por estado

In [19]:
(
    pnad
    .select('renda',  'uf', 'sexo')
    .groupby('uf', 'sexo').agg(avg('renda').alias('media'))
    .orderBy(f.col('uf').asc())
    .toPandas()
)

,uf,sexo,media
0,Acre,Homem,2278.415679
1,Acre,Mulher,1999.182491
2,Alagoas,Homem,1373.689157
3,Alagoas,Mulher,1319.560923
4,Amapá,Homem,2329.527638
5,Amapá,Mulher,2439.692456
6,Amazonas,Mulher,1633.377941
7,Amazonas,Homem,1641.451171
8,Bahia,Mulher,1377.833516
9,Bahia,Homem,1400.507972


## 6 - Qual é a média de renda para as pessoas na região Sul do país, que estão na força de trabalho e possuem entre 25 e 35 anos? 

In [20]:
#Nosso dataframe não possui coluna com a informação da região.
#Vamos criar um função que adicione essa informação ao nosso dataframe

def regiao_udf(uf):
    if uf in('Acre','Pará','Rondônia', 'Roraima', 'Tocantins'):
        return 'Norte'
    elif uf in('Maranhão','Paraíba','Pernambuco','Piauí','Rio Grande do Norte'):
        return 'Nordeste'
    elif uf in('Goiás','Mato Grosso do Sul','Mato Grosso'):
        return 'Centro-Oeste'
    elif uf in('Espírito Santo','Minas Gerais','Rio de Janeiro','São Paulo'):
        return 'Sudeste'
    elif uf in('Rio Grande do Sul', 'Santa Catarina', 'Paraná'):
        return 'Sul'
    elif uf in('Amazonas', 'Amapá'):
        return 'Norte'
    elif uf in ('Alagoas', 'Bahia', 'Ceará'):
        return 'Nordeste'
    else:
        'Vazio'

udf_function = udf(regiao_udf, StringType())


In [21]:
#Agora aplicamos a função no nosso dataframe
pnad = pnad.withColumn('regiao', udf_function(pnad.uf))

In [22]:
pnad.head(5)

[Row(_id='600ce0685daa95c6616007b1', ano=2020, trimestre=3, uf='Acre', sexo='Mulher', idade=35, cor='Parda', graduacao='null', trab='Pessoas na força de trabalho', ocup='Pessoas ocupadas', renda=800.0, horastrab=36.0, anosesco=16, regiao='Norte'),
 Row(_id='600ce0685daa95c6616007cc', ano=2020, trimestre=3, uf='Acre', sexo='Homem', idade=49, cor='Parda', graduacao='null', trab='Pessoas na força de trabalho', ocup='Pessoas ocupadas', renda=9500.0, horastrab=40.0, anosesco=15, regiao='Norte'),
 Row(_id='600ce0685daa95c6616007bf', ano=2020, trimestre=3, uf='Acre', sexo='Mulher', idade=30, cor='Branca', graduacao='null', trab='Pessoas na força de trabalho', ocup='Pessoas ocupadas', renda=1049.0, horastrab=44.0, anosesco=16, regiao='Norte'),
 Row(_id='600ce0685daa95c6616007af', ano=2020, trimestre=3, uf='Acre', sexo='Homem', idade=47, cor='Preta', graduacao='null', trab='Pessoas na força de trabalho', ocup='Pessoas ocupadas', renda=1000.0, horastrab=30.0, anosesco=10, regiao='Norte'),
 Row(_

In [23]:
#Finalmente podemos responder a questão
(
    pnad
    .select('renda', 'regiao', 'idade')
    .filter(((f.col('idade') >= 25) & (f.col('idade') <= 35)))
    .filter(f.col('regiao') == 'Sul')
    .groupby('regiao').agg(avg('renda').alias('media'))
    .orderBy(f.col('media').desc())
    .toPandas()
)

,regiao,media
0,Sul,2458.175551


# Qual a renda média da região Sudeste

In [24]:
(
    pnad
    .select('renda', 'regiao')
    .filter(f.col('regiao') == 'Sudeste')
    .groupby('regiao').agg(avg('renda').alias('media'))
    .orderBy(f.col('media').desc())
    .toPandas()
)

,regiao,media
0,Sudeste,2474.693993
